# John Vincent Deniega
# ADS 509 - Fall 2024
# 29 September 2024

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

Note: This notebook was initiated via a template from the University of San Diego prompt from ADS 509 Module 4 assignment 1.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

from nltk.corpus import stopwords
from string import punctuation
punctuation = set(punctuation)
# Stopwords
sw = stopwords.words("english")
stopwords = set(sw)

# Functions from USD's ADS509 Assignment 3's template
def remove_punctuation(text, punct_set=punctuation) : 
    return("".join([ch for ch in text if ch not in punct_set]))
def remove_stop(tokens) :
    return [word for word in tokens if word not in stopwords]

# Automate the cleaning and tokenization process for any text
def clean_token(text):
    text = ''.join([i for i in text if not i.isdigit()]) # Filter number characters
    text = text.lower() # force to lowercase
    text = remove_punctuation(text)
    tokens = text.split()
    tokens = remove_stop(tokens)
    return ' '.join(tokens)

In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

#### Explore SQL data structure

In [3]:
# Set up SQLite connection
#conv_db_path = "Desktop/usd_ADS/github/ads509-mod4/2020_Conventions.db"
conn = convention_db

# Get the table names from the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = conn.execute(query).fetchall()
tables

[('conventions',)]

In [4]:
# Get the column names from the 'conventions' table
# Other table structure options can be found in
# https://database.guide/4-ways-to-get-information-about-a-tables-structure-in-sqlite/
query_columns = "PRAGMA table_info(conventions);"
columns = conn.execute(query_columns).fetchall()

# Show head / top 5 for testing simplicity
query_head = "SELECT * FROM conventions LIMIT 5;"
head = conn.execute(query_head).fetchall()
columns,head

([(0, 'party', 'TEXT', 0, None, 0),
  (1, 'night', 'INTEGER', 0, None, 0),
  (2, 'speaker', 'TEXT', 0, None, 0),
  (3, 'speaker_count', 'INTEGER', 0, None, 0),
  (4, 'time', 'TEXT', 0, None, 0),
  (5, 'text', 'TEXT', 0, None, 0),
  (6, 'text_len', 'TEXT', 0, None, 0),
  (7, 'file', 'TEXT', 0, None, 0)],
 [('Democratic',
   4,
   'Unknown',
   1,
   '00:00',
   'Skip to content The Company Careers Press Freelancers Blog × Services Transcription Captions Foreign Subtitles Translation Freelancers About Contact Login « Return to Transcript Library home  Transcript Categories  All Transcripts 2020 Election Transcripts Classic Speech Transcripts Congressional Testimony & Hearing Transcripts Debate Transcripts Donald Trump Transcripts Entertainment Transcripts Financial Transcripts Interview Transcripts Political Transcripts Press Conference Transcripts Speech Transcripts Sports Transcripts Technology Transcripts Aug 21, 2020 2020 Democratic National Convention (DNC) Night 4 Transcript Rev  ›

#### Finding: At this point, we have found that the database has eight columns with counting starting at zero.

In [5]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

# Limit to 10 when testing code functionality
# Remove when performing full analysis on corpus
query_results = convention_cur.execute(
                            '''
                            SELECT * FROM conventions;
                            ''')

for row in query_results :
    party = row[0] # Extract the first row as the party
    text = clean_token(row[5]) # Extract text from "sixth" row
    convention_data.append([text, party]) # Order as cleaned text then party

In [6]:
convention_data[:5]

[['skip content company careers press freelancers blog × services transcription captions foreign subtitles translation freelancers contact login « return transcript library home transcript categories transcripts election transcripts classic speech transcripts congressional testimony hearing transcripts debate transcripts donald trump transcripts entertainment transcripts financial transcripts interview transcripts political transcripts press conference transcripts speech transcripts sports transcripts technology transcripts aug democratic national convention dnc night transcript rev › blog › transcripts › election transcripts › democratic national convention dnc night transcript night democratic national convention dnc august read full transcript event transcribe content try rev free save time transcribing captioning subtitling',
  'Democratic'],
 ['i’m calling full session th quadrennial national convention democratic party order welcome final session historic memorable convention we’

Let's look at some random entries and see if they look right. 

In [7]:
random.choices(convention_data,k=10)

[['bully school marched street knocked door dr', 'Democratic'],
 ['fear live understandably somehow tomorrow said “no insurance you’re covered” devastating hey laura',
  'Democratic'],
 ['', 'Republican'],
 ['california', 'Democratic'],
 ['that’s ran office', 'Democratic'],
 ['threat nation democracy real it’s clear it’s present we’ve watched president three years look he’s instilling fear mean joking instilling fear showing division stroking racial division undercutting every institution designed check abuse power president anyone else reason order solidify base expand power',
  'Democratic'],
 ['west virginia', 'Republican'],
 ['also moved joe work going back forth leader wrote violence women act enacted assault weapons ban vice president implemented recovery act brought country back great recessions championed affordable care act protecting millions americans preexisting conditions spent decades promoting american values interests around world',
  'Democratic'],
 ['defenders life in

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [8]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2341 as features in the model.


In [9]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    words = text.split() # Split on whitespace by default
    ret_dict = dict() # Initialize dictionary
    for word in words: # Iterate through all tokens (on whitespace) from the given "text"
        if word in fw and word not in ret_dict: # Ensure token is only set True on first read
            ret_dict[word] = True
    
    return(ret_dict)

#### Testing with assert statement on conv_features function

In [10]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [11]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [12]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [13]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.496


In [14]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting? It appears that the classifier on its own using the training and test set on a single word or single n-gram still only provides a roughly 50/50 accuracy on two labels. This suggests that this method may be no better than a random guesser.

### My Observations

In addition to the above observation, it is notable that 23 of the top 25 most informative features are Republican words ranging from 25.8 to 1 to 10.3 to 1. This is suggests that Republican text key words are far more emphasized relative to the corpus than that of the top words in the Democratic text. This also suggests that the inverse is true in that these words are negative indicators of the opposite party. For example, if the word "china" appears in a text, then is both an indicator that it is strongly Republican, but also an indicator that the text is very unlikely Democratic. The key word, "votes" would apply in the opposite manner.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [15]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [16]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

#### Verify results are listed as expected

In [17]:
results[:5]

[('Mo Brooks',
  'Republican',
  b'"Brooks Joins Alabama Delegation in Voting Against Flawed Funding Bill" http://t.co/3CwjIWYsNq'),
 ('Mo Brooks',
  'Republican',
  b'"Brooks: Senate Democrats Allowing President to Give Americans\xe2\x80\x99 Jobs to Illegals" #securetheborder https://t.co/mZtEaX8xS6'),
 ('Mo Brooks',
  'Republican',
  b'"NASA on the Square" event this Sat. 11AM \xe2\x80\x93 4PM. Stop by &amp; hear about the incredible work done in #AL05! @DowntownHSV http://t.co/R9zY8WMEpA'),
 ('Mo Brooks',
  'Republican',
  b'"The trouble with Socialism is that eventually you run out of other people\'s money." - Margaret Thatcher https://t.co/X97g7wzQwJ'),
 ('Mo Brooks',
  'Republican',
  b'"The trouble with socialism is eventually you run out of other people\'s money" \xe2\x80\x93 Thatcher. She\'ll be sorely missed. http://t.co/Z8gBnDQUh8')]

In [18]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results:
    party = row[1] # Parse that row's party
    tweet = row[2].decode('utf-8') # Parse the tweet from the "third" row
    tweet_data.append([tweet, party]) # Add this row's entry tuple as a list

In [19]:
tweet_data[:5] # Check the first five like a head(5) function call

[['"Brooks Joins Alabama Delegation in Voting Against Flawed Funding Bill" http://t.co/3CwjIWYsNq',
  'Republican'],
 ['"Brooks: Senate Democrats Allowing President to Give Americans’ Jobs to Illegals" #securetheborder https://t.co/mZtEaX8xS6',
  'Republican'],
 ['"NASA on the Square" event this Sat. 11AM – 4PM. Stop by &amp; hear about the incredible work done in #AL05! @DowntownHSV http://t.co/R9zY8WMEpA',
  'Republican'],
 ['"The trouble with Socialism is that eventually you run out of other people\'s money." - Margaret Thatcher https://t.co/X97g7wzQwJ',
  'Republican'],
 ['"The trouble with socialism is eventually you run out of other people\'s money" – Thatcher. She\'ll be sorely missed. http://t.co/Z8gBnDQUh8',
  'Republican']]

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [20]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

### Repeat the word cutoff step from conventions for tweet data

In [21]:
word_cutoff = 5

tokens = [w for t, p in tweet_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 80481 as features in the model.


### Repeat the train, test split from conventions for tweet data

In [22]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in tweet_data]
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.656


In [23]:
for tweet, party in tweet_data_sample :
    # Get the estimated party using the classifier
    # Reference
    # https://www.nltk.org/api/nltk.classify.naivebayes.html
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: Earlier today, I spoke on the House Floor abt protecting health care for women and praised @PPmarmonte for their work on the Central Coast. https://t.co/WqgTRzT7VV
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: Go Tribe! #RallyTogether https://t.co/0NXutFL9L5
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: Apparently, Trump thinks it's just too easy for students overwhelmed by the crushing burden of debt to pay off student loans #TrumpBudget https://t.co/ckYQO5T0Qh
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: We’re grateful for our first responders, our rescue personnel, our firefighters, our police, and volunteers who have been working tirelessly to keep people safe, provide much-needed help, while putting their own lives on the line.

https://t.co/eZPv0vMIz3
Actual party is Republican and our classifer says Democratic.

Here'

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [24]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party (Note, copy-pasted right side like above)
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [25]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 2393, 'Democratic': 1885}),
             'Democratic': defaultdict(int,
                         {'Republican': 329, 'Democratic': 5395})})

### Reflections

Based on the results, given Republican tweets, the classifier was 55.94 percent accurate. This is in stark contrast given Democratic tweets where the classifer was 94.25 percent accurate. Further exploration into the data is needed to identify why there's such a disparity in performance by party affiliation. Initial impressions suggest that Republican tweets may be more politically ambiguous relative to those of their Democratic collegues, which may be a result of party tendencies to use the given medium, Twitter. Further, given the accuracy of Democratic tweets by the Naive Bayes classifier, it may suggest that there's a strong set of feature words that are largely indicative of Democratic party affiliation. Finally, there may be an opportunity to use the key words from the convention data based on most important features to further refine this model.